<a href="https://colab.research.google.com/github/LitonSarker/3D-Deep-Learning/blob/main/PointTransformerV3_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Create a separate Python 3.10 environment inside the notebook and run PTv3 from that interpreter.

# Install micromamba (user-space conda)

%%bash
set -e
cd /content
curl -L -o micromamba.tar.bz2 https://micro.mamba.pm/api/micromamba/linux-64/latest
tar -xvjf micromamba.tar.bz2 bin/micromamba
chmod +x /content/bin/micromamba
echo 'done'


bin/micromamba
done


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3935    0  3935    0     0   2367      0 --:--:--  0:00:01 --:--:-- 3842k
100 6491k  100 6491k    0     0  2461k      0  0:00:02  0:00:02 --:--:-- 24.4M


In [2]:
# Create Python 3.10 env

%%bash
set -e
/export/content/bin/micromamba --version || /content/bin/micromamba --version

# create env
/content/bin/micromamba create -y -n ptv3py310 -c conda-forge python=3.10 pip

# show python
/content/bin/micromamba run -n ptv3py310 python -c "import sys; print(sys.version)"


2.5.0


Transaction

  Prefix: /root/.local/share/mamba/envs/ptv3py310

  Updating specs:

   - python=3.10
   - pip


  Package              Version  Build                 Channel          Size
─────────────────────────────────────────────────────────────────────────────
  Install:
─────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex          0.1  conda_forge           conda-forge       3kB
  + _openmp_mutex          4.5  2_gnu                 conda-forge      24kB
  + bzip2                1.0.8  hda65f42_8            conda-forge     260kB
  + ca-certificates   2026.1.4  hbd8a1cb_0            conda-forge     147kB
  + icu                   78.2  h33c6efd_0            conda-forge      13MB
  + ld_impl_linux-64      2.45  default_hbd61a6d_105  conda-forge     731kB
  + libexpat             2.7.3  hecca717_0            conda-forge      77kB
  + libffi               3.5.2  h9ec8514_0            conda-forge      58kB
  + libgcc              15.2.

bash: line 2: /export/content/bin/micromamba: No such file or directory


In [3]:
# Install Torch (cu118) inside that env

%%bash
set -e
/content/bin/micromamba run -n ptv3py310 pip install -q --upgrade pip

/content/bin/micromamba run -n ptv3py310 pip install -q \
  --index-url https://download.pytorch.org/whl/cu118 \
  torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2+cu118


In [4]:
# Install spconv wheel + deps (no JIT)

%%bash
set -e
/content/bin/micromamba run -n ptv3py310 pip install -q spconv-cu118==2.3.8
/content/bin/micromamba run -n ptv3py310 pip install -q "timm>=0.9.16,<1.0" einops addict pyyaml termcolor plyfile numpy

# verify spconv imports WITHOUT building
/content/bin/micromamba run -n ptv3py310 python -c "import spconv; print('spconv ok:', spconv.__version__)"


spconv ok: 2.3.8


In [5]:
# Clone PTv3

%%bash
set -e
cd /content
rm -rf PointTransformerV3
git clone --depth 1 https://github.com/Pointcept/PointTransformerV3.git
ls PointTransformerV3 | head


assets
LICENSE
model.py
Pointcept
README.md
serialization


Cloning into 'PointTransformerV3'...


In [ ]:
# Upload your PLY(s)

from google.colab import files
uploaded = files.upload()
ply_files = list(uploaded.keys())
ply_files


In [ ]:
# Fix NumPy 2.x issue (pin NumPy < 2)

%%bash
set -e
/content/bin/micromamba run -n ptv3py310 pip install -q "numpy<2"


In [ ]:
# Install torch_scatter matching torch 2.2.2 + cu118

%%bash
set -e
/content/bin/micromamba run -n ptv3py310 pip install -q \
  torch-scatter -f https://data.pyg.org/whl/torch-2.2.2+cu118.html


In [ ]:
# Quick verify (optional but recommended)

%%bash
set -e
/content/bin/micromamba run -n ptv3py310 python - << 'PY'
import numpy as np
import torch
import torch_scatter
print("numpy:", np.__version__)
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("torch_scatter:", torch_scatter.__version__)
PY


In [ ]:
# Run PTv3 forward pass on your real XYZ-only PLY(s)

%%bash
set -e
cd /content

# IMPORTANT: run with py310 env
/content/bin/micromamba run -n ptv3py310 python - << 'PY'
import os
os.environ["SPCONV_DISABLE_JIT"] = "1"
os.environ["CUMM_DISABLE_JIT"] = "1"

import numpy as np
import torch
from plyfile import PlyData
from PointTransformerV3.model import PointTransformerV3


device = "cuda"
print("torch:", torch.__version__, "cuda:", torch.version.cuda, "gpu:", torch.cuda.get_device_name(0))

# list uploaded ply files in /content (Colab upload path)
import glob
ply_files = sorted(glob.glob("/content/*.ply"))
print("PLYs:", ply_files)

def run_one(ply_path, grid_size=0.02, max_points=20000):
    ply = PlyData.read(ply_path)
    v = ply["vertex"].data
    xyz = np.stack([v["x"], v["y"], v["z"]], axis=1).astype(np.float32)

    if xyz.shape[0] > max_points:
        idx = np.random.choice(xyz.shape[0], max_points, replace=False)
        xyz = xyz[idx]

    N = xyz.shape[0]

    # PTv3 signature defaults to in_channels=6 -> use XYZ + dummy(3)
    feat = np.concatenate([xyz, np.ones((N, 3), dtype=np.float32)], axis=1)

    coord_t  = torch.from_numpy(xyz).to(device)
    feat_t   = torch.from_numpy(feat).to(device)
    offset_t = torch.tensor([N], device=device, dtype=torch.int32)

    batch = {"coord": coord_t, "feat": feat_t, "offset": offset_t, "grid_size": float(grid_size)}

    model = PointTransformerV3(
        in_channels=6,
        enable_flash=False,
        enc_patch_size=(128,128,128,128,128),
        dec_patch_size=(128,128,128,128),
    ).to(device).eval()

    with torch.no_grad():
        out = model(batch)

    return out, N

for f in ply_files:
    out, N = run_one(f)
    base = os.path.splitext(os.path.basename(f))[0]
    save_name = f"/content/{base}_ptv3_out.pt"
    torch.save(out, save_name)
    print("\nOK:", base, "N=", N, "saved:", save_name)

PY


In [ ]:
# Download outputs

from google.colab import files
import glob
for p in glob.glob("/content/*_ptv3_out.pt"):
    files.download(p)
